# Understanding Homelessness Rates

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

## Introduction
### Key Homelessness Issues
- What is homelessness and definition challenges
- Causes of homelessness
- Homelessness classifications including chronic, sheltered/un

### Point in Time Counts
- History of counts
- Methodological issues
- Rationale for category inclusions

### Data Background
- Originally was going to use a dataset from Kaggle but decided to pull straight from HUD-CoC site. 
- Used [2007 - 2017 PIT Counts by State](https://www.hudexchange.info/resource/3031/pit-and-hic-data-since-2007/) and converted to single database

(borrowed from: https://www.kaggle.com/bltxr9/eda-of-total-homeless-population)
This dataset was generated by CoC and provided to HUD. Note: HUD did not conduct a full data quality review on the data submitted by each CoC.

What is the [Continuum of Care (CoC) Program](https://www.hudexchange.info/programs/coc/)?

Original Data: [PIT and HIC Data Since 2007](https://www.hudexchange.info/resource/3031/pit-and-hic-data-since-2007/)

CoC-HUD Summary Reports: [CoC Homeless Populations and Subpopulations Reports](https://www.hudexchange.info/programs/coc/coc-homeless-populations-and-subpopulations-reports/)

**Other Resources**

[Funding Awards](https://www.hudexchange.info/programs/coc/awards-by-component/)

[CoC Dashboard Reports](https://www.hudexchange.info/programs/coc/coc-dashboard-reports/)

[CoC Housing Inventory Count Reports](https://www.hudexchange.info/programs/coc/coc-housing-inventory-count-reports/)

## Data Wrangling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Inspect Data

In [20]:
df = pd.read_csv('homeless-pit-by-state.csv')
df.head()

,State,Year,Number of CoCs,Total Homeless,Sheltered Homeless,Unsheltered Homeless,Homeless Individuals,Sheltered Homeless Individuals,Unsheltered Homeless Individuals,Homeless People in Families,...,Unsheltered Parenting Youth (Under 25),Parenting Youth Under 18,Sheltered Parenting Youth Under 18,Unsheltered Parenting Youth Under 18,Parenting Youth Age 18-24,Sheltered Parenting Youth Age 18-24,Unsheltered Parenting Youth Age 18-24,Children of Parenting Youth,Sheltered Children of Parenting Youth,Unsheltered Children of Parenting Youth
0,AK,2017,2,"1,845","1,551",294,"1,354","1,060",294,491,...,0,0,0,0,22,22,0,39,39,0
1,AL,2017,8,"3,793","2,656","1,137","2,985","1,950","1,035",808,...,3,6,6,0,23,20,3,39,35,4
2,AR,2017,6,"2,467","1,273","1,194","2,068",937,"1,131",399,...,0,0,0,0,10,10,0,13,13,0
3,AZ,2017,3,"8,947","5,781","3,166","6,488","3,423","3,065","2,459",...,0,0,0,0,81,81,0,112,112,0
4,CA,2017,43,"134,278","42,636","91,642","112,756","25,022","87,734","21,522",...,234,16,11,5,874,645,229,"1,058",782,276


#### Check Data Types and Missing Data

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 605 entries, 0 to 604
Data columns (total 45 columns):
State                                                          605 non-null object
Year                                                           605 non-null int64
Number of CoCs                                                 605 non-null int64
Total Homeless                                                 605 non-null object
Sheltered Homeless                                             605 non-null object
Unsheltered Homeless                                           605 non-null object
Homeless Individuals                                           605 non-null object
Sheltered Homeless Individuals                                 605 non-null object
Unsheltered Homeless Individuals                               605 non-null object
Homeless People in Families                                    605 non-null object
Sheltered Homeless People in Families                          605 

_Observations_
- Missing data is consistent across groups of categories. Visual inspection of data confirms that this is due to additional categories added in subsequent years. 
- Data is all in object format and will need to be converted to float. 
- Not all categories are multually exclusive, confirmation is required to confirm how data is summed.

Visualization of data that all columns are available from 2015 onwards, 2011 - 2014 contains columns up to Unsheltered Homeless Veterans and 2007 - 2013 contains columns up to Unsheltered Chronically Homeless Individuals.

#### Check unique values

In [10]:
df[['State', 'Year']].nunique()

State    56
Year     11
dtype: int64

_Observations_
- There are 11 years of data contained in the set (2007 - 2017)
- Need to confirm what states are covered within state
- Not worried about unique values for the other columns

In [13]:
df['State'].unique()

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD',
       'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
       'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC',
       'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY',
       'KS*'], dtype=object)

- It appears that the list includes some US territories and DC. I am less familiar with state abbreviations - will need to get state names for reference.
- One of states is KS*, note from dataset says: The number of CoCs in 2017 was 399. However, MO-604 merged in 2016 and covers territory in both MO and KS, contributing to the PIT count in both states. This will need to be inspected individually to understand.

### Clean Data
#### Convert Coumns to `float`

In [64]:
df.columns

Index(['State', 'Year', 'Number of CoCs', 'Total Homeless',
       'Sheltered Homeless', 'Unsheltered Homeless', 'Homeless Individuals',
       'Sheltered Homeless Individuals', 'Unsheltered Homeless Individuals',
       'Homeless People in Families', 'Sheltered Homeless People in Families',
       'Unsheltered Homeless People in Families', 'Chronically Homeless',
       'Sheltered Chronically Homeless', 'Unsheltered Chronically Homeless',
       'Chronically Homeless Individuals',
       'Sheltered Chronically Homeless Individuals',
       'Unsheltered Chronically Homeless Individuals',
       'Chronically Homeless People in Families',
       'Sheltered Chronically Homeless People in Families',
       'Unsheltered Chronically Homeless People in Families',
       'Homeless Veterans', 'Sheltered Homeless Veterans',
       'Unsheltered Homeless Veterans',
       'Homeless Unaccompanied Youth (Under 25)',
       'Sheltered Homeless Unaccompanied Youth (Under 25)',
       'Unsheltered Home

It was discovered that missing data for states was reported as `.` For example:

In [97]:
df.query('State == "MP"')[:5]

,State,Year,Number of CoCs,Total Homeless,Sheltered Homeless,Unsheltered Homeless,Homeless Individuals,Sheltered Homeless Individuals,Unsheltered Homeless Individuals,Homeless People in Families,...,Unsheltered Parenting Youth (Under 25),Parenting Youth Under 18,Sheltered Parenting Youth Under 18,Unsheltered Parenting Youth Under 18,Parenting Youth Age 18-24,Sheltered Parenting Youth Age 18-24,Unsheltered Parenting Youth Age 18-24,Children of Parenting Youth,Sheltered Children of Parenting Youth,Unsheltered Children of Parenting Youth
26,MP,2017,1,672,24,648,208,11,197,464,...,0,0,0,0,0,0,0,0,0,0
81,MP,2016,0,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
136,MP,2015,0,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
191,MP,2014,0,.,.,.,.,.,.,.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246,MP,2013,0,.,.,.,.,.,.,.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


These needed to be converted to NaN to allow for further data transformation.

In [21]:
df.replace('.', np.NaN, inplace=True)
df.query('State == "MP"')[:5]

,State,Year,Number of CoCs,Total Homeless,Sheltered Homeless,Unsheltered Homeless,Homeless Individuals,Sheltered Homeless Individuals,Unsheltered Homeless Individuals,Homeless People in Families,...,Unsheltered Parenting Youth (Under 25),Parenting Youth Under 18,Sheltered Parenting Youth Under 18,Unsheltered Parenting Youth Under 18,Parenting Youth Age 18-24,Sheltered Parenting Youth Age 18-24,Unsheltered Parenting Youth Age 18-24,Children of Parenting Youth,Sheltered Children of Parenting Youth,Unsheltered Children of Parenting Youth
26,MP,2017,1,672,24,648,208,11,197,464,...,0,0,0,0,0,0,0,0,0,0
81,MP,2016,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136,MP,2015,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,MP,2014,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246,MP,2013,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 605 entries, 0 to 604
Data columns (total 45 columns):
State                                                          605 non-null object
Year                                                           605 non-null int64
Number of CoCs                                                 605 non-null int64
Total Homeless                                                 595 non-null object
Sheltered Homeless                                             595 non-null object
Unsheltered Homeless                                           595 non-null object
Homeless Individuals                                           595 non-null object
Sheltered Homeless Individuals                                 595 non-null object
Unsheltered Homeless Individuals                               595 non-null object
Homeless People in Families                                    595 non-null object
Sheltered Homeless People in Families                          595 

This increased the amount of missing data for the columns - there is need to further examine the patterns of missing data. It is suggested that the most valuable method of investigation will be a year by year examination of what is missing.

All commas needed to be removed from the str values to allow conversion to `float` while managing `NaN` values.

In [22]:
df[df.columns] = df[df.columns].replace({',':''}, regex = True)

In [27]:
df[df.columns[3:]] = df[df.columns[3:]].astype(float)
df.head()

,State,Year,Number of CoCs,Total Homeless,Sheltered Homeless,Unsheltered Homeless,Homeless Individuals,Sheltered Homeless Individuals,Unsheltered Homeless Individuals,Homeless People in Families,...,Unsheltered Parenting Youth (Under 25),Parenting Youth Under 18,Sheltered Parenting Youth Under 18,Unsheltered Parenting Youth Under 18,Parenting Youth Age 18-24,Sheltered Parenting Youth Age 18-24,Unsheltered Parenting Youth Age 18-24,Children of Parenting Youth,Sheltered Children of Parenting Youth,Unsheltered Children of Parenting Youth
0,AK,2017,2,1845.0,1551.0,294.0,1354.0,1060.0,294.0,491.0,...,0.0,0.0,0.0,0.0,22.0,22.0,0.0,39.0,39.0,0.0
1,AL,2017,8,3793.0,2656.0,1137.0,2985.0,1950.0,1035.0,808.0,...,3.0,6.0,6.0,0.0,23.0,20.0,3.0,39.0,35.0,4.0
2,AR,2017,6,2467.0,1273.0,1194.0,2068.0,937.0,1131.0,399.0,...,0.0,0.0,0.0,0.0,10.0,10.0,0.0,13.0,13.0,0.0
3,AZ,2017,3,8947.0,5781.0,3166.0,6488.0,3423.0,3065.0,2459.0,...,0.0,0.0,0.0,0.0,81.0,81.0,0.0,112.0,112.0,0.0
4,CA,2017,43,134278.0,42636.0,91642.0,112756.0,25022.0,87734.0,21522.0,...,234.0,16.0,11.0,5.0,874.0,645.0,229.0,1058.0,782.0,276.0


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 605 entries, 0 to 604
Data columns (total 45 columns):
State                                                          605 non-null object
Year                                                           605 non-null int64
Number of CoCs                                                 605 non-null int32
Total Homeless                                                 595 non-null float64
Sheltered Homeless                                             595 non-null float64
Unsheltered Homeless                                           595 non-null float64
Homeless Individuals                                           595 non-null float64
Sheltered Homeless Individuals                                 595 non-null float64
Unsheltered Homeless Individuals                               595 non-null float64
Homeless People in Families                                    595 non-null float64
Sheltered Homeless People in Families                       

This confirms that all data has been adjusted as desired.

#### Confirm Column Configurations
To confirm how each of the individual columns are grouped, the values were summed and then tested.

In [32]:
test = df.iloc[:, 3:].sum()
test

Total Homeless                                                 6634879.0
Sheltered Homeless                                             4288711.0
Unsheltered Homeless                                           2346168.0
Homeless Individuals                                           4185343.0
Sheltered Homeless Individuals                                 2260177.0
Unsheltered Homeless Individuals                               1925166.0
Homeless People in Families                                    2449536.0
Sheltered Homeless People in Families                          2028534.0
Unsheltered Homeless People in Families                         421002.0
Chronically Homeless                                           1164058.0
Sheltered Chronically Homeless                                  439453.0
Unsheltered Chronically Homeless                                724605.0
Chronically Homeless Individuals                                888346.0
Sheltered Chronically Homeless Individuals         

A series of tests were completed to confirm the expected combinations:

`Total Homeless` = `Sheltered Homeless` + `Unsheltered Homeless`

In [40]:
(test['Sheltered Homeless'] + test['Unsheltered Homeless']) == test['Total Homeless']

True

`Homeless Individuals` = `Sheltered Homeless Individuals` + `Unsheltered Homeless Individuals`

In [41]:
(test['Sheltered Homeless Individuals'] + test['Unsheltered Homeless Individuals']) == test['Homeless Individuals']

True

`Homeless People in Families` = `Sheltered Homeless People in Families` + `Unsheltered Homeless People in Families`

In [42]:
(test['Sheltered Homeless People in Families'] + test['Unsheltered Homeless People in Families']) == test['Homeless People in Families']

True

`Total Homeless` = `Homeless Individuals` + `Homeless People in Families`

In [43]:
(test['Homeless Individuals'] + test['Homeless People in Families']) == test['Total Homeless']

True

`Chronically Homeless` = `Sheltered Chronically Homeless` + `Unsheltered Chronically Homeless`

In [44]:
(test['Sheltered Chronically Homeless'] + test['Unsheltered Chronically Homeless']) == test['Chronically Homeless']

True

It is also expected that `Chronically Homeless` is a subset of `Total Homeless` and so the values for `Chronically Homeless` should be less than that of the total.

In [45]:
test['Chronically Homeless'] < test['Total Homeless']

True

As it is not captured, a column for `Not Chronically Homeless` should be added.

`Sheltered Chronically Homeless` = `Sheltered Chronically Homeless Individuals` + `Sheltered Chronically Homeless People in Families`

In [47]:
(test['Sheltered Chronically Homeless Individuals'] + test['Sheltered Chronically Homeless People in Families']) == test['Sheltered Chronically Homeless']

False

In [48]:
print(test['Sheltered Chronically Homeless Individuals'] + test['Sheltered Chronically Homeless People in Families'])
print(test['Sheltered Chronically Homeless'])

428331.0
439453.0


This is an unexpected result. Need to look into where the inequalities lie.

In [53]:
not_equal = (df['Sheltered Chronically Homeless Individuals'] + df['Sheltered Chronically Homeless People in Families']) != df['Sheltered Chronically Homeless']
df[['Year', 'State', 'Sheltered Chronically Homeless Individuals', 'Sheltered Chronically Homeless People in Families', 'Sheltered Chronically Homeless']][not_equal]

,Year,State,Sheltered Chronically Homeless Individuals,Sheltered Chronically Homeless People in Families,Sheltered Chronically Homeless
81,2016,MP,NaN,NaN,NaN
136,2015,MP,NaN,NaN,NaN
191,2014,MP,NaN,NaN,NaN
246,2013,MP,NaN,NaN,NaN
301,2012,MP,NaN,NaN,NaN
356,2011,MP,NaN,NaN,NaN
385,2010,AK,270.0,NaN,119.0
386,2010,AL,911.0,NaN,502.0
387,2010,AR,247.0,NaN,145.0
388,2010,AZ,1052.0,NaN,544.0


In all cases where it does not match, no information has been reported for `Sheltered Chronically Homeless People in Families`. This suggests that the CoCs in the state do not have the ability to identify people as in a family. Typically, the number of `Sheltered Chronically Homeless Individuals` is less than `Sheltered Chronically Homeless People in Families`. But in some cases it is not.

In [61]:
ind_greater = df['Sheltered Chronically Homeless Individuals'] > df['Sheltered Chronically Homeless']
results_greater = df[['Year', 'State', 'Sheltered Chronically Homeless Individuals', 'Sheltered Chronically Homeless']][ind_greater]
results_greater

,Year,State,Sheltered Chronically Homeless Individuals,Sheltered Chronically Homeless
385,2010,AK,270.0,119.0
386,2010,AL,911.0,502.0
387,2010,AR,247.0,145.0
388,2010,AZ,1052.0,544.0
390,2010,CO,853.0,510.0
396,2010,GU,5.0,1.0
397,2010,HI,176.0,96.0
398,2010,IA,207.0,182.0
399,2010,ID,128.0,37.0
401,2010,IN,633.0,478.0


All of these results occurred between 2007 and 2010. It suggests that there may have been some errors in counting methodology in some CoCs and it looks as if this was corrected for 2011 onwards. This suggests that interpretation of `Sheltered Chronically Homeless` before 2010 may not be accurate. It maybe worth investigating how CoC/HUD interpreted the results for these years from applicable states.

Want to check how many states are involved for each year.

In [58]:
print(results_greater.query('Year == 2007').shape[0])
print(results_greater.query('Year == 2008').shape[0])
print(results_greater.query('Year == 2009').shape[0])
print(results_greater.query('Year == 2010').shape[0])

19
17
21
28


The number of states where this occurred is not consistent over the years, but may also reflect states correcting their methodology and additional states reporting. 

`Unsheltered Chronically Homeless` = `Unsheltered Chronically Homeless Individuals` + `Unsheltered Chronically Homeless People in Families`

In [59]:
(test['Unsheltered Chronically Homeless Individuals'] + test['Unsheltered Chronically Homeless People in Families']) == test['Unsheltered Chronically Homeless']

False

It appears that a similar issue to what was happening with the `Sheltered Chronically Homeless` numbers is happening with the `Unsheltered Chronically Homeless` numbers.

In [60]:
ind_greater = df['Unsheltered Chronically Homeless Individuals'] > df['Unsheltered Chronically Homeless']
results_greater = df[['Year', 'State', 'Unsheltered Chronically Homeless Individuals', 'Unsheltered Chronically Homeless People in Families', 'Unsheltered Chronically Homeless']][ind_greater]
results_greater

,Year,State,Unsheltered Chronically Homeless Individuals,Unsheltered Chronically Homeless People in Families,Unsheltered Chronically Homeless
399,2010,ID,111.0,NaN,74.0
402,2010,KS,51.0,NaN,42.0
413,2010,MT,106.0,NaN,49.0
415,2010,ND,5.0,NaN,3.0
438,2010,WV,183.0,NaN,158.0
454,2009,ID,76.0,NaN,53.0
457,2009,KS,60.0,NaN,42.0
468,2009,MT,78.0,NaN,53.0
500,2008,CO,693.0,NaN,633.0
509,2008,ID,53.0,NaN,35.0


Interestingly, there are far fewer instances where the number of individuals is greater than the total count of people who were `Unsheltered Chronically Homeless` as compared with `Sheltered Chronically Homeless` numbers. Again, the discrepancies only occur between 2007 to 2010. 

Will need to examine further to decide how to address these numbers.

`Homeless Veterans` = `Sheltered Homeless Veterans` + `Unsheltered Homeless Vetereans`

In [62]:
(test['Sheltered Homeless Veterans'] + test['Unsheltered Homeless Veterans']) == test['Homeless Veterans']

True

It is expected that `Homeless Veterans` is a subset of `Total Homeless` and should be lower.

In [63]:
test['Homeless Veterans'] < test['Total Homeless']

True

As it is not captured, a column for `Homeless Non-Veteran` should be added.

`Homeless Unaccompanied Youth (Under 25)` = `Sheltered Homeless Unaccompanied Youth (Under 25)` + `Unsheltered Homeless Unaccompanied Youth (Under 25)`

In [64]:
(test['Sheltered Homeless Unaccompanied Youth (Under 25)'] + test['Unsheltered Homeless Unaccompanied Youth (Under 25)']) == test['Homeless Unaccompanied Youth (Under 25)']

True

It is expected that `Homeless Unaccompanied Youth (Under 25)` is a subset of `Homeless Individuals` and should be lower.

In [65]:
test['Homeless Unaccompanied Youth (Under 25)'] < test['Homeless Individuals']

True

As it is not captured, a column for `Homeless Adult` should be added.

`Homeless Unaccompanied Youth (Under 25)` = `Homeless Unaccompanied Children (Under 18)` + `Homeless Unaccompanied Young Adults (Age 18-24)`

In [66]:
(test['Homeless Unaccompanied Children (Under 18)'] + test['Homeless Unaccompanied Young Adults (Age 18-24)']) == test['Homeless Unaccompanied Youth (Under 25)']

True

`Homeless Unaccompanied Children (Under 18)` = `Sheltered Homeless Unaccompanied Children (Under 18)` + `Unsheltered Homeless Unaccompanied Children (Under 18)`

In [67]:
(test['Sheltered Homeless Unaccompanied Children (Under 18)'] + test['Unsheltered Homeless Unaccompanied Children (Under 18)']) == test['Homeless Unaccompanied Children (Under 18)']

True

`Homeless Unaccompanied Young Adults (Age 18 - 24)` = `Sheltered Homeless Unaccompanied Young Adults (Age 18 - 24)` + `Unsheltered Homeless Unaccompanied Young Adults (Age 18 - 24)`

In [68]:
(test['Sheltered Homeless Unaccompanied Young Adults (Age 18-24)'] + test['Unsheltered Homeless Unaccompanied Young Adults (Age 18-24)']) == test['Homeless Unaccompanied Young Adults (Age 18-24)']

True

`Parenting Youth (Under 25)` = `Sheltered Parenting Youth (Under 25)` + `Unsheltered Parenting Youth (Under 25)`

In [69]:
(test['Sheltered Parenting Youth (Under 25)'] + test['Unsheltered Parenting Youth (Under 25)']) == test['Parenting Youth (Under 25)']

True

`Parenting Youth (Under 25)` = `Parenting Youth Under 18` + `Parenting Youth Age 18-24`

In [71]:
(test['Parenting Youth Under 18'] + test['Parenting Youth Age 18-24']) == test['Parenting Youth (Under 25)']

True

`Parenting Youth Age 18-24` = `Sheltered Parenting Youth Age 18-24` + `Unsheltered Parenting Youth Age 18-24`

In [73]:
(test['Sheltered Parenting Youth Age 18-24'] + test['Unsheltered Parenting Youth Age 18-24']) == test['Parenting Youth Age 18-24']

True

`Parenting Youth Under 18` = `Sheltered Parenting Youth Under18` + `Unsheltered Parenting Youth Under 18`

In [74]:
(test['Sheltered Parenting Youth Under 18'] + test['Unsheltered Parenting Youth Under 18']) == test['Parenting Youth Under 18']

True

`Children of Parenting Youth` = `Sheltered Children of Parenting Youth` + `Unsheltered Children of Parenting Youth`

In [75]:
(test['Sheltered Children of Parenting Youth'] + test['Unsheltered Children of Parenting Youth']) == test['Children of Parenting Youth']

True

Children of parenting youth and their parents create a subset of `Homeless People in Families` and their sum should be less than this. 

In [76]:
(test['Parenting Youth (Under 25)'] + test['Children of Parenting Youth']) < test['Homeless People in Families']

True

This is considered sufficient comparisons to confirm that the data is structured in the way intended. In all cases, except those of Chronic Homelessness, the data configuration aligned with expectations. 

It is noted that there are a number of different ways that data can be divided by on factors of sheltering type (sheltered or unsheltered), homelessness type (chronic or not), family status (individual or family), veteran status (veteran or not), age (under 25 or not), but not all segmentations are carried across each category.

The groupings are as follows.